In [ ]:
# %% [markdown]
### 1. Import Libraries
%%
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind
import warnings
warnings.filterwarnings('ignore')

# %% [markdown]
### 2. Load Data
%%
# Load batting data
batting = pd.read_csv('fact_bating_summary.csv')
# Fix column name typo
batting.rename(columns={'teamInnings': 'team', 'battingPos': 'batting_position'}, inplace=True)

# Load bowling data
bowling = pd.read_csv('fact_bowling_summary.csv')

# Load player dimension
players = pd.read_csv('dim_players.csv')

# %% [markdown]
### 3. Data Aggregation & Noise Filtering
%%
# Aggregate batting stats per player
batting_agg = batting.groupby('batsmanName').agg(
    total_innings=('match_id', 'nunique'),
    total_runs=('runs', 'sum'),
    total_balls=('balls', 'sum'),
    avg_strike_rate=('SR', 'mean')
).reset_index()

# Aggregate bowling stats per player
bowling_agg = bowling.groupby('bowlerName').agg(
    total_innings_bowled=('match_id', 'nunique'),
    total_wickets=('wickets', 'sum'),
    avg_economy=('economy', 'mean')
).reset_index()

# Merge with player data
df = pd.merge(players, batting_agg, 
              left_on='name', 
              right_on='batsmanName', 
              how='left')

df = pd.merge(df, bowling_agg,
              left_on='name',
              right_on='bowlerName',
              how='left')

# %% [markdown]
### 4. Filtering Criteria
%%
# Filter players with ≥3 innings batted/bowled
df_filtered = df[
    (df['total_innings'].fillna(0) >= 3) |
    (df['total_innings_bowled'].fillna(0) >= 3)
]

# Remove extreme strike rate outliers
df_filtered = df_filtered[
    (df_filtered['avg_strike_rate'] <= 200) | 
    (df_filtered['avg_strike_rate'].isna())
]

# %% [markdown]
### 5. Hypothesis Testing: Openers Analysis
%%
# Prepare opener data
openers = df_filtered[
    (df_filtered['playingRole'].str.contains('Top order', na=False)) &
    (df_filtered['avg_strike_rate'] > 140)
]

non_openers = df_filtered[
    ~df_filtered['name'].isin(openers['name'])
]

# Independent t-test
t_stat, p_value = ttest_ind(
    openers['total_runs'].dropna(),
    non_openers['total_runs'].dropna(),
    equal_var=False
)

print(f"Openers with SR>140 vs Others:\nT-statistic: {t_stat:.2f}, P-value: {p_value:.4f}")

# %% [markdown]
### 6. Power BI Export
%%
# Save cleaned data
df_filtered.to_csv('cleaned_player_data.csv', index=False)

# Create role-specific datasets
openers_df = df_filtered[df_filtered['playingRole'].str.contains('Top order', na=False)]
anchors_df = df_filtered[df_filtered['playingRole'].str.contains('Middle order', na=False)]

openers_df.to_csv('openers_analysis.csv', index=False)
anchors_df.to_csv('anchors_analysis.csv', index=False)